In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

#from urllib import urlretrieve
#import cPickle as pickle
import os
#import gzip

import numpy as np
import theano

import lasagne
from lasagne import layers
from lasagne.updates import adam

#from nolearn.lasagne import NeuralNet
#from nolearn.lasagne import visualize

#from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix

In [7]:
def contracting_block(layer):
    #ugotovi num_filters, padding
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(3, 3, 3), stride=2, pad=1)
    tmp_layer = layer
    #ugotovi primeren batch norm layer
    layer = layers.BatchNormLayer(layer)
    #ugotovi primeren prelu layer
    layer = layers.prelu(layer)
    #ugotovi num_filters, padding
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(3, 3, 3), stride=1, pad=1)
    layer = layers.ElemwiseSumLayer([layer, tmp_layer])
    #print(theano.tensor.shape(lasagne.layers.get_output(layer)))
    layer = layers.batch_norm(layer)
    layer = layers.prelu(layer)
    return layer
    

In [3]:
def start(input):
    layer = layers.Conv3DLayer(input, num_filters=1, filter_size=(3, 3, 3), stride=1, pad=1)
    #print(theano.tensor.shape(lasagne.layers.get_output(layer)))
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    return layer

In [4]:
def expanding_block(layer):
    # dodaj feature merging block
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(1, 1, 1), stride=1, pad=1)
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    # kako deluje deconvolutional layer --> DEKONVOLUCIJA
    #layer = layers.Conv3DLayer(input, num_filters=1, filter_size=(3, 3, 3), stride=0.5)
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(1, 1, 1), stride=1, pad=1)
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    return layer

In [40]:
def binary_jaccard_index(predictions,targets):
    intersection = theano.tensor.minimum(predictions, targets)
    union = theano.tensor.maximum(predictions, targets)
    axes = tuple(range(1,5))
    return intersection.sum(axis=axes) / union.sum(axis=axes)

In [5]:
#load the data

data = np.load('data-150-150-20.npy')
seg_data = np.load('segdata-150-150-20.npy')

data = np.expand_dims(data, axis=1)
seg_data = np.expand_dims(seg_data, axis=1)

train_data = data[:-2]
validation_data = data[-2:]
train_seg = seg_data[:-2]
validation_seg = seg_data[-2:]

for x in train_seg:
    print(x.shape)



(1L, 20L, 150L, 150L)
(1L, 20L, 150L, 150L)
(1L, 20L, 150L, 150L)
(1L, 20L, 150L, 150L)
(1L, 20L, 150L, 150L)
(1L, 20L, 150L, 150L)
(1L, 20L, 150L, 150L)
(1L, 20L, 150L, 150L)


In [6]:
input_var = theano.tensor.tensor5()
target_var = theano.tensor.tensor5()


input = lasagne.layers.InputLayer((None, 1, 20, 150, 150), input_var)
layer = start(input)
layer = contracting_block(layer)
layer = expanding_block(layer)

prediction = lasagne.layers.get_output(layer)
#uporabiti pravi loss metric
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean() #+ 1e-4 * lasagne.regularization.regularize_network_params(network, lasagne.regularization.l2)

#print(loss.eval())

# create parameter update expressions
params = lasagne.layers.get_all_params(layer, trainable=True)
updates = lasagne.updates.adam(loss, params, learning_rate=0.01)

# compile training function that updates parameters and returns training loss
train_fn = theano.function([input_var, target_var], loss, updates=updates)

# train network (assuming you've got some training data in numpy arrays)
for epoch in range(100):
    loss = train_fn(train_data, train_seg)
    print("Epoch %d: Loss %g" % (epoch + 1, loss))

ValueError: Input dimension mis-match. (input[0].shape[2] = 20, input[1].shape[2] = 14)
Apply node that caused the error: Elemwise{mul,no_inplace}(<TensorType(float64, 5D)>, Elemwise{log,no_inplace}.0)
Toposort index: 300
Inputs types: [TensorType(float64, 5D), TensorType(float64, (False, True, False, False, False))]
Inputs shapes: [(8L, 1L, 20L, 150L, 150L), (8L, 1L, 14L, 79L, 79L)]
Inputs strides: [(3600000L, 3600000L, 180000L, 1200L, 8L), (698992L, 698992L, 49928L, 632L, 8L)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Sum{axis=[4], acc_dtype=float64}(Elemwise{mul,no_inplace}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "C:\Anaconda2\lib\site-packages\ipykernel\kernelbase.py", line 390, in execute_request
    user_expressions, allow_stdin)
  File "C:\Anaconda2\lib\site-packages\ipykernel\ipkernel.py", line 196, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "C:\